In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, normalized_mutual_info_score
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, v_measure_score, silhouette_score, davies_bouldin_score, mutual_info_score
from sklearn.manifold import TSNE
from sklearn.covariance import EmpiricalCovariance

from mpl_toolkits.mplot3d import Axes3D
from yellowbrick.cluster import KElbowVisualizer
from collections import Counter
from scipy.spatial.distance import pdist, squareform, mahalanobis
from numpy.linalg import inv, pinv
from kneed import KneeLocator
import warnings



# Task 1 Data Preprocessing and Exploratory Data Analysis

We perform the following steps:
1. Load the dataset ("Dataset.csv") and verify its integrity.
2. Confirm that there are no missing values.
3. Identify and analyze outliers using visualizations such as boxplots.
4. Visualize feature distributions with histograms and KDE plots to understand the
overall distribution of each feature.
5. Review feature statistics (e.g., mean, standard deviation) to get insights into the
data.
6. Normalize or standardize the dataset so that all features contribute equally in
distance calculations, which is crucial for clustering.

### Subtask 1: Load the dataset ("Dataset.csv") and verify its integrity.

Manual inspection of the dataset determined that there are 900 rows (excluding the header row) and 8 columns. There to satisfy the integrity requirement we take that to mean the row and column counts are equal after the dataframe is loaded.







In [ ]:
########
# Subtask 1 - Load the dataset ("Dataset.csv") and verify its integrity.
#
# Purpose: To load the required data from Dataset.csv into a pandas dataframe called 'df' and then to verify its 
#           integrity. Integrity is understood to mean that the correct number of rows and columns have been 
#           loaded that are in agreement with the manual inspection of these counts. 
#           If the row or column count is not in agreement with the manual count then the assertion will fail.
# Takeaway: The datasets integrity has been validation otherwise execution will be stopped.
########
df = pd.read_csv("Dataset.csv") # load the dataset
rows, cols = df.shape # get the row and column counts
print(f"Dataset shape: {rows} rows, {cols} columns") 

# programmatic verification of the integrity of the dataset, throw an error if the row or column counts are not equal to 900 and 8 respectively
if rows != 900:
    assert False, "The number of rows in the dataset is not equal to 900"
if cols != 8:
    assert False, "The number of columns in the dataset is not equal to 8"

print("Dataset integrity verified")

### Subtask 2: Confirm that there are no missing values.
Count the number of missing values in each column and throw an error if any are found.

In [ ]:
########
# Subtask 2 - Confirm that there are no missing values.
# 
# Purpose: To ensure that there are no missing values anywhere in the dataset. 
#          If there are missing values then the assertion will fail.
# Takeaway: The dataset has no missing values otherwise execution will be stopped.
########
missing_values_count = df.isnull().sum()
if missing_values_count.sum() > 0:
    assert False, "The dataset contains missing values!!!! FIX"
print("Good, No missing values")

### Subtask 3: Identify and analyze outliers using visualizations such as boxplots.
Boxplots for each numerical feature to identify and analyze outliers. Calculate and display statistics about potential outliers. This can be done by calculating the IQR and then using that to identify the lower and upper bounds of the outliers.
The label is categorical so not included in outlier detection.


In [ ]:
########
# Subtask 3 - Identify and analyze outliers using visualizations such as boxplots.
#
# Purpose: Outliers may occur in numerical features. This subtask is to identify and analyse these outliers.
#          Boxplots are used to identify and analyse outliers. The label is categorical so not included in outlier detection.
# Takeaway: A boxplot per numerical feature is created and displayed. This allows for the manual inspection of the outliers.
#          Outliers in the plots are identified as points that are outside of the whiskers of the boxplot. It can be seen from the plots
#          that there are outliers in ever feature, but either above or below the upper or lower whiskers respectively.        
########
sns.set_palette('viridis') # set colour scheme

# Get numerical features from the dataset (only numerical features are considered for outlier detection)
numerical_features = df.select_dtypes(include=[np.number]).columns

# Create boxplots for each numerical feature
plt.figure(figsize=(16, 10))
for i, feature in enumerate(numerical_features):
    plt.subplot(3, 3, i+1)  # Adjust grid based on number of features
    sns.boxplot(y=df[feature])
    plt.title(f'Boxplot of {feature}')
    plt.tight_layout()

plt.suptitle('Boxplots for Numerical Features to Identify Outliers', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()


### Subtask 4: Visualise feature distributions with histograms and KDE plots to understand the overall distribution of each feature.

Seaborn has differing functions for histograms and KDE plots. Use these.

In [ ]:
########
# Subtask 4: Visualise feature distributions with histograms and KDE plots to understand the overall distribution of each feature.
#
# Purpose: To visualise the distribution of each feature in the dataset. Histograms and KDE plots are used to visualise the distribution of each feature.
# Takeaway: A histogram and KDE plot per numerical feature was created and displayed. It showed that ever numerical feature is skewed. The skew direction for each feature is:
#           - Area: Right
#           - MajorAxisLength: Right
#           - MinorAxisLength: Right
#           - Eccentricity: Left
#           - ConvexArea: Right
#           - Extent: Left
#           - Perimeter: Right
########


plt.figure(figsize=(16, 12))


numerical_features = df.select_dtypes(include=[np.number]).columns # list of numerical features

# Display histograms for each numerical feature
for i, feature in enumerate(numerical_features):
    plt.subplot(3, 3, i+1)  # Adjust grid based on number of features
    sns.histplot(df[feature])
    plt.title(f'Histogram of {feature}')
    plt.tight_layout()

plt.suptitle('Feature Distributions with Histograms', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

# Create KDE plots for each numerical feature
plt.figure(figsize=(16, 12))

# Get numerical features from the dataset
numerical_features = df.select_dtypes(include=[np.number]).columns

# Create KDE plots for each numerical feature
for i, feature in enumerate(numerical_features):
    plt.subplot(3, 3, i+1)  # Adjust grid based on number of features
    sns.kdeplot(df[feature], fill=True)
    plt.title(f'KDE Plot of {feature}')
    plt.tight_layout()

plt.suptitle('Feature Distributions with KDE Plots', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()





All features are skewed to either the left or right

### Subtask 5 - Review feature statistics (e.g., mean, standard deviation) to get insights into the data.

In [ ]:
########
# Subtask 5 - Review feature statistics (e.g., mean, standard deviation) to get insights into the data.
#
# Purpose: To review the statistics of the numerical features in the dataset.
# Takeaway: From the descriptive statistics it can be observed that:
#           - Area: Right skewed as mean is greater than the median (50% percentile)
#           - MajorAxisLength: Right skewed as mean is greater than the median (50% percentile)
#           - MinorAxisLength: Right skewed as mean is greater than the median (50% percentile)
#           - Eccentricity: Left skewed as mean is less than the median (50% percentile)
#           - ConvexArea: Right skewed as mean is greater than the median (50% percentile)
#           - Extent: Left skewed as mean is less than the median (50% percentile)
#           - Perimeter: Right skewed as mean is greater than the median (50% percentile)
#           Standard deviations are quite large for Area, ConvexArea, MajorAxisLength, MinorAxisLength, Perimeter. This indicates there are a large range of objects in the dataset
########

print("Basic Statistics for Numerical Features via Pandas Dataframe describe:")
display(df.describe())

# Calculate additional statistics that aren't in describe()
print("\nAdditional Statistics:")
numerical_stats = pd.DataFrame({
    'Median': df.select_dtypes(include=[np.number]).median(),
    'Skewness': df.select_dtypes(include=[np.number]).skew(),
    'Kurtosis': df.select_dtypes(include=[np.number]).kurt(),
    'IQR': df.select_dtypes(include=[np.number]).quantile(0.75) - df.select_dtypes(include=[np.number]).quantile(0.25),
    'Range': df.select_dtypes(include=[np.number]).max() - df.select_dtypes(include=[np.number]).min()
})
display(numerical_stats)

# Generate a correlation matrix
print("\nCorrelation Matrix:")
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
display(correlation_matrix)

# Plot the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.show()


It can be seen that lengths and areas are highly correlated, which is expected as area is a function of length.

### Subtask 6 - Normalize or standardize the dataset so that all features contribute equally in distance calculations, which is crucial for clustering.

For every numeric feature, we will normalize it to a range of 0 to 1.

In [ ]:
########
# Subtask 6 - Normalize or standardize the dataset so that all features contribute equally in distance calculations, which is crucial for clustering.
#
# Purpose: Normalise the dataset so that all features contribute equally in distance calculations.
# Takeaway: The dataset was normalized to a range of 0 to 1 for every numeric feature.
########
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist() # create list of numerical columns
scaler = MinMaxScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns]) # fit then transform the numerical columns

df.head()


# Task 2 - Impact of the Number of Clusters on KMeans Clustering with Euclidean Distance

The subtask for this are:
1. Apply KMeans clustering (using Euclidean distance) on the standardized dataset.
2. For a range of cluster numbers (e.g., from 1 to 10), compute the inertia (SSE) and plot
these values to identify the “elbow” point.

In [ ]:
########
# Task 2 - Impact of the Number of Clusters on KMeans Clustering with Euclidean Distance
#
# Purpose: Apply KMeans clustering on the normalised dataset and to determine the optimal number of cluster using the elbow method.
# Takeaway: The elbow appears to be when the cluster number is 3. The KElbowVisualizer is a handy library that displays the elbow point.
########

model = KMeans()
visualizer = KElbowVisualizer(
    model, k=(1,11), metric='distortion', timings=False
) #distortion same as Euclidean distance

visualizer.fit(df[numerical_columns])        # Fit the data to the visualizer
visualizer.show() 



From the above plot, the elbow appears to be when the cluster number is 5 as after that point the inertia decreases at a slower rate than for lower cluster numbers.

# Task 3 - Evaluating the Stability of KMeans and KMeans++ Initialization

Subtasks are:
1. Run KMeans clustering 50 times using two initialization methods:
    - Standard random initialization.
    - KMeans++ initialization.
2. Compute and compare the average inertia (SSE) and the Silhouette Score for each
method over these iterations.

In [ ]:
##########
# Task 3 - Evaluating the Stability of KMeans and KMeans++ Initialization
#
# Purpose: To compare the KMeans and Kmeans++ with regard to their stability when initialised randomly in 50 different runs.
# Takeaway: KMeans++ is slightly more susceptible to differences in initialisation values. 
#           This can be observed from the distribution of the inertia and silhouette scores with KMeans++ distribution being more dispersed.
##########

n_iterations = 50
n_clusters = 3  # Using 5 clusters based on the elbow method from previous task
random_inertias = []
random_silhouette_scores = []
kmeans_plus_inertias = []
kmeans_plus_silhouette_scores = []

for i in range(n_iterations): # iterate 50 times

    # random init kmeans
    kmeans_random = KMeans(n_clusters=n_clusters, init='random', random_state=i)
    kmeans_random.fit(df[numerical_columns])
    random_inertias.append(kmeans_random.inertia_)
    
    # random init silhouette score
    labels_random = kmeans_random.labels_
    random_silhouette_scores.append(silhouette_score(df[numerical_columns], labels_random))
    
    # KMeans++ initialisation
    kmeans_plus = KMeans(n_clusters=n_clusters, init='k-means++', random_state=i)
    kmeans_plus.fit(df[numerical_columns])
    kmeans_plus_inertias.append(kmeans_plus.inertia_)
    
    # calc silhouette score for kmeans++
    labels_plus = kmeans_plus.labels_
    kmeans_plus_silhouette_scores.append(silhouette_score(df[numerical_columns], labels_plus))

# calc average metrics
avg_random_inertia = np.mean(random_inertias)
avg_random_silhouette = np.mean(random_silhouette_scores)
avg_kmeans_plus_inertia = np.mean(kmeans_plus_inertias)
avg_kmeans_plus_silhouette = np.mean(kmeans_plus_silhouette_scores)

# calc standard deviations 
std_random_inertia = np.std(random_inertias)
std_random_silhouette = np.std(random_silhouette_scores)
std_kmeans_plus_inertia = np.std(kmeans_plus_inertias)
std_kmeans_plus_silhouette = np.std(kmeans_plus_silhouette_scores)

# Display results
print("Standard Random Initialisation:")
print(f"Average Inertia: {avg_random_inertia:.2f} (±{std_random_inertia:.2f})")
print(f"Average Silhouette Score: {avg_random_silhouette:.4f} (±{std_random_silhouette:.4f})")
print("\nKMeans++ Initialisation:")
print(f"Average Inertia: {avg_kmeans_plus_inertia:.2f} (±{std_kmeans_plus_inertia:.2f})")
print(f"Average Silhouette Score: {avg_kmeans_plus_silhouette:.4f} (±{std_kmeans_plus_silhouette:.4f})")

# Plot the distribution of inertias for both methods
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(random_inertias, alpha=0.7, label='Random Init')
plt.hist(kmeans_plus_inertias, alpha=0.7, label='KMeans++ Init')
plt.xlabel('Inertia')
plt.ylabel('Frequency')
plt.title('Distribution of Inertia Values')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(random_silhouette_scores, alpha=0.7, label='Random Init')
plt.hist(kmeans_plus_silhouette_scores, alpha=0.7, label='KMeans++ Init')
plt.xlabel('Silhouette Score')
plt.ylabel('Frequency')
plt.title('Distribution of Silhouette Scores')
plt.legend()

plt.tight_layout()
plt.show()


As can be seen from the above, kmeans++ is slightly more susceptible to differences in initialisation values.


# Task 4 - Clustering Evaluation Using Purity and Mutual Information

Subtasks are:

1. Use KMeans (with the optimal k from Question 2) to cluster the data. Assume the
dataset contains a ground-truth label column (e.g., "label"). For each cluster, assign a
label based on the majority class.
2. Evaluation Metrics: Compute and report the following:
    1. Purity Score: Measures how homogeneous each cluster is relative to the true
labels.
    2. Mutual Information Score: Quantifies the mutual dependence between the
clustering results and the true labels.
    3. Silhouette Score: Evaluates the clustering quality without reference to the
ground truth by comparing intra-cluster cohesion versus inter-cluster
separation.

In [ ]:
#########
# Task 4 - Clustering Evaluation Using Purity and Mutual Information
#
# Purpose: To evaluate the clustering results using purity, mutual information and silhouette score.
# Takeaway: The purity score of 0.84 is quite high, indicating that the members of each cluster are quite homogeneous and in agreement with the true labels.
#           The mutual information score of 0.3343 is not very high indicating that the structure of the clusters is not very similar to the true labels.
#           The silhouette score of 0.3372 is not very high indicating that the clusters are not very well separated. 
#           Although the cluster purity was high, the clustering results were not very good most likely due to splitting of the Besni class into two clusters.
#########


# 3 was the optimal k from task 2
k = 3

# get labels and features
X = df.drop('label', axis=1)
true_labels = df['label']

# KMeans with the optimal k
kmeans = KMeans(n_clusters=k, random_state=42, init='k-means++')
cluster_labels = kmeans.fit_predict(X)

# based on majority class assign a label to each cluster
cluster_to_label = {}
for cluster_id in range(k):
    cluster_indices = np.where(cluster_labels == cluster_id)[0]
    cluster_true_labels = true_labels.iloc[cluster_indices]
    # get the most common label
    most_common_label = Counter(cluster_true_labels).most_common(1)[0][0]
    cluster_to_label[cluster_id] = most_common_label

# print mappings
print("Cluster to Label Mapping:")
for cluster_id, label in cluster_to_label.items():
    print(f"Cluster {cluster_id} has label: {label}")

# calc purity score
def purity_score(y_true, y_pred):
    contingency_matrix = np.zeros((k, len(np.unique(y_true))))
    
    for i in range(len(y_true)):
        true_label_idx = np.where(np.unique(y_true) == y_true.iloc[i])[0][0]
        contingency_matrix[y_pred[i], true_label_idx] += 1
    
    return np.sum(np.max(contingency_matrix, axis=1)) / len(y_true)

# calc evaluation metrics
purity = purity_score(true_labels, cluster_labels)
mutual_info = normalized_mutual_info_score(true_labels, cluster_labels)
silhouette = silhouette_score(X, cluster_labels)

print("\nEvaluation Metrics:")
print(f"Purity Score: {purity:.4f}")
print(f"Normalized Mutual Information Score: {mutual_info:.4f}")
print(f"Silhouette Score: {silhouette:.4f}")

# Graph clusters
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
for cluster_id in range(k):
    cluster_points = X[cluster_labels == cluster_id]
    plt.scatter(cluster_points.iloc[:, 0], cluster_points.iloc[:, 1], 
                label=f"Cluster {cluster_id}: {cluster_to_label[cluster_id]}")

plt.title('Clusters by KMeans')
plt.xlabel(X.columns[0])
plt.ylabel(X.columns[1])
plt.legend()

plt.subplot(1, 2, 2)
for label in np.unique(true_labels):
    label_points = X[true_labels == label]
    plt.scatter(label_points.iloc[:, 0], label_points.iloc[:, 1], label=label)

plt.title('True Labels')
plt.xlabel(X.columns[0])
plt.ylabel(X.columns[1])
plt.legend()

plt.tight_layout()
plt.show()


# Task 5 Principal Component Analysis (PCA) for Dimensionality Reduction

Subtasks are:
1. Apply PCA to reduce the dataset to 4 principal components.
2. Plot the cumulative variance explained by the principal components and determine
how many components are needed to retain 90% of the total variance.
3. Create a 3D scatter plot of the first three principal components.

In [ ]:
######
# Task 5 - Principal Component Analysis (PCA) for Dimensionality Reduction
#
# Purpose: To reduce the dataset to 4 principal components, to plot the cumulative variance explained by the principal components and to determine the number of components needed to retain 90% of the total variance.
# Takeaway: PCA was used to reduce the dataset to 4 principal components. From this it was determined that PC1, PC2 and PC3 were needed to retain 90% of the total variance. (3 components were needed). The amount of variance explained by each component is as follows:
#           PC1: 0.6903 (0.6903 cumulative)
#           PC2: 0.2076 (0.8979 cumulative)
#           PC3: 0.0898 (0.9877 cumulative)
#           PC4: 0.0081 (0.9958 cumulative)
######


X_std = StandardScaler().fit_transform(X)

# Subtask 1

# Apply PCA with 4 components
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_std)

pca_df = pd.DataFrame(
    data=X_pca,
    columns=['PC1', 'PC2', 'PC3', 'PC4']
)

pca_df['label'] = true_labels



# Subtask 2 - Graph the explained variance ratio
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, label='Individual explained variance')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Cumulative explained variance')
plt.axhline(y=0.9, color='r', linestyle='--', label='90% threshold')

# Calc how many components needed for 90% variance
components_needed = np.argmax(cumulative_variance >= 0.9) + 1
plt.axvline(x=components_needed, color='g', linestyle='--', 
            label=f'{components_needed} components needed for 90% variance')

plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Principal Components')
plt.legend()
plt.grid(True)
plt.show()

print(f"\nExplained variance ratio by component:")
for i, var in enumerate(explained_variance):
    print(f"PC{i+1}: {var:.4f} ({cumulative_variance[i]:.4f} cumulative)")

print(f"\nNumber of components needed to retain 90% variance: {components_needed}")

# Make a 3D scatter plot of the first three principal components
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

unique_labels = np.unique(true_labels)
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))

for label, color in zip(unique_labels, colors):
    mask = pca_df['label'] == label
    ax.scatter(
        pca_df.loc[mask, 'PC1'],
        pca_df.loc[mask, 'PC2'],
        pca_df.loc[mask, 'PC3'],
        label=label,
        color=color,
        alpha=0.7,
        s=50
    )

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('3D Projection of First Three Principal Components')
ax.legend()

plt.tight_layout()
plt.show()


# Task 6 - Density Based Clustering Using DBSCAN with Different Distance Metrics

Subtasks are:

1. Apply DBSCAN to the dataset twice:
    1. Once using Euclidean distance.
    2. Once using Mahalanobis distance.
2. Determine the optimal values for eps (ε) and min_samples for each distance metric.
3. Compare the clustering results from both distance metrics.

In [ ]:

#######
# Task 6 - Density Based Clustering Using DBSCAN with Different Distance Metrics
#
# Purpose: To apply DBSCAN to the dataset twice: once using Euclidean distance and once using Mahalanobis distance.
# Takeaway: Care had to be taken when using DBSCAN as the number of clusters to be used cannot be set beforehand. As we know there are two label ideally we would have two clusters. 
#            DBSCAN also assigns a -1 label to noise points which had to be filtered out when calculating the silhouette score.
#            The best euclidean parameters were eps=0.65, min_samples=5. The best mahalanobis parameters were eps=7.00, min_samples=5.
#            The cluster evaluation metrics were as follows:
#           Euclidean - Purity: 0.5522, Mutual Information: 0.0148, Silhouette: 0.2529876685755327, Cluster Sizes: 3
#           Mahalanobis - Purity: 0.5022, Mutual Information: 0.0009, Silhouette: 0.6096798484416615, Cluster Sizes: 3
#           The Euclidean distance had a better clustering result as it had a higher purity and mutual information score. The mahalanobis distance had a higher silhouette score which indicated that the clusters were more separated.
#######

# Calc purity
def calculate_purity(true_labels, predicted_labels):
    contingency_matrix = np.zeros((len(np.unique(true_labels)), len(np.unique(predicted_labels))))
    
    for i, true_label in enumerate(np.unique(true_labels)):
        for j, pred_label in enumerate(np.unique(predicted_labels)):
            contingency_matrix[i, j] = np.sum((true_labels == true_label) & (predicted_labels == pred_label))
    
    cluster_sizes = np.sum(contingency_matrix, axis=0)
    max_correct = np.sum(np.max(contingency_matrix, axis=0))
    
    purity = max_correct / len(true_labels)
    return purity



# find optimal eps for euclidean distance
def find_optimal_eps(X, min_samples, target_clusters=2, eps_range=np.arange(0.1, 2.0, 0.05)):
    results = []
    
    for eps in eps_range:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        noise_points = np.sum(labels == -1)
        
        results.append({
            'eps': eps,
            'n_clusters': n_clusters,
            'noise_points': noise_points,
            'labels': labels
        })
    
    return min(results, key=lambda x: abs(x['n_clusters'] - target_clusters))

# min samples to try
min_samples_values = [5, 10, 15, 20]
euclidean_results = []

for min_samples in min_samples_values:
    result = find_optimal_eps(X_std, min_samples)
    result['min_samples'] = min_samples
    euclidean_results.append(result)
    print(f"Euclidean - min_samples={min_samples}, eps={result['eps']:.2f}, clusters={result['n_clusters']}, noise points={result['noise_points']}")

# choose best eps
best_euclidean = min(euclidean_results, key=lambda x: abs(x['n_clusters'] - 2))
print(f"\nBest Euclidean parameters: eps={best_euclidean['eps']:.2f}, min_samples={best_euclidean['min_samples']}")

# distances for mahalanobis
cov = np.cov(X_std.T)
inv_cov = np.linalg.inv(cov)

#  custom distance matrix using Mahalanobis distance
def create_mahalanobis_distance_matrix(X, inv_cov):
    n = X.shape[0]
    dist_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            dist = mahalanobis(X[i], X[j], inv_cov)
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
    
    return dist_matrix

# create Mahalanobis distance matrix
mahalanobis_dist_matrix = create_mahalanobis_distance_matrix(X_std, inv_cov)

# find optimal eps for Mahalanobis distance
def find_optimal_eps_precomputed(dist_matrix, min_samples, target_clusters=2, eps_range=np.arange(0.5, 10.0, 0.5)):
    results = []
    
    for eps in eps_range:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
        labels = dbscan.fit_predict(dist_matrix)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        noise_points = np.sum(labels == -1)
        
        results.append({
            'eps': eps,
            'n_clusters': n_clusters,
            'noise_points': noise_points,
            'labels': labels
        })
    
    return min(results, key=lambda x: abs(x['n_clusters'] - target_clusters))

mahalanobis_results = []

for min_samples in min_samples_values:
    result = find_optimal_eps_precomputed(mahalanobis_dist_matrix, min_samples)
    result['min_samples'] = min_samples
    mahalanobis_results.append(result)
    print(f"Mahalanobis - min_samples={min_samples}, eps={result['eps']:.2f}, clusters={result['n_clusters']}, noise points={result['noise_points']}")

# best parameters for mahalanobis
best_mahalanobis = min(mahalanobis_results, key=lambda x: abs(x['n_clusters'] - 2))
print(f"\nBest Mahalanobis parameters: eps={best_mahalanobis['eps']:.2f}, min_samples={best_mahalanobis['min_samples']}")

# dbscan with best parameters for euclidean
euclidean_dbscan = DBSCAN(eps=best_euclidean['eps'], min_samples=best_euclidean['min_samples'])
euclidean_labels = euclidean_dbscan.fit_predict(X_std)

# dbscan with best parameters for mahalanobis
mahalanobis_dbscan = DBSCAN(eps=best_mahalanobis['eps'], min_samples=best_mahalanobis['min_samples'], metric='precomputed')
mahalanobis_labels = mahalanobis_dbscan.fit_predict(mahalanobis_dist_matrix)

# metrics for Euclidean distance
euclidean_purity = calculate_purity(true_labels, euclidean_labels)
euclidean_mi = mutual_info_score(true_labels, euclidean_labels)

# filter out noise cluster (-1 label)
euclidean_filtered_indices = euclidean_labels != -1
euclidean_filtered_labels = euclidean_labels[euclidean_filtered_indices]
true_labels_euclidean_filtered = true_labels[euclidean_filtered_indices]

# silhouette score
if len(set(euclidean_filtered_labels)) > 1:
    euclidean_silhouette = silhouette_score(X_std[euclidean_filtered_indices], euclidean_filtered_labels)
else:
    euclidean_silhouette = "N/A"  # Not applicable if only one cluster remains

# metrics for Mahalanobis distance
mahalanobis_purity = calculate_purity(true_labels, mahalanobis_labels)
mahalanobis_mi = mutual_info_score(true_labels, mahalanobis_labels)

# filter out noise cluster (-1 label)
mahalanobis_filtered_indices = mahalanobis_labels != -1
mahalanobis_filtered_labels = mahalanobis_labels[mahalanobis_filtered_indices]
true_labels_mahalanobis_filtered = true_labels[mahalanobis_filtered_indices]

if len(set(mahalanobis_filtered_labels)) > 1:
    mahalanobis_silhouette = silhouette_score(X_std[mahalanobis_filtered_indices], mahalanobis_filtered_labels)
else:
    mahalanobis_silhouette = "N/A"  # Not applicable if only one cluster remains

euclidean_cluster_size = len(set(euclidean_labels))
mahalanobis_cluster_size = len(set(mahalanobis_labels))

print("\nMetrics results:")
print(f"Euclidean - Purity: {euclidean_purity:.4f}, Mutual Information: {euclidean_mi:.4f}, Silhouette: {euclidean_silhouette}, Cluster Sizes: {euclidean_cluster_size}")
print(f"Mahalanobis - Purity: {mahalanobis_purity:.4f}, Mutual Information: {mahalanobis_mi:.4f}, Silhouette: {mahalanobis_silhouette}, Cluster Sizes: {mahalanobis_cluster_size}")



# Task 7 - Clustering Performance on PCA-Reduced v Full Dataset

1. Apply KMeans clustering to:
    1. The original standardized dataset.
    2. The PCA-transformed dataset (using the principal components from Question5).
2.  Evaluate the clustering quality using the Silhouette Score.
3. Compare whether the PCA-transformed dataset results in better-separated and more compact clusters relative to the full dataset.

In [ ]:
#########
# Task 7 - Clustering Performance on PCA-Reduced v Full Dataset
#
# Purpose: To apply KMeans clustering to the original and PCA-transformed datasets and to evaluate the clustering quality using the Silhouette Score.
# Takeaway: The results for the two datasets were:
#           Original Dataset: k=2, Silhouette Score=0.441
#PCA Dataset: k=2, Silhouette Score=0.484
#########


# function to run kmeans and eval silhouette and inertia score
def apply_kmeans_and_evaluate(data, name, n_clusters_range=range(2, 11)):
    results = []
    
    for k in n_clusters_range:
        # Apply KMeans
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(data)
        
        # Calculate silhouette score
        sil_score = silhouette_score(data, cluster_labels)
        
        # Calculate inertia (sum of squared distances to closest centroid)
        inertia = kmeans.inertia_
        
        results.append({
            'k': k,
            'silhouette': sil_score,
            'inertia': inertia,
            'labels': cluster_labels
        })
        
        print(f"{name} with {k} clusters - Silhouette Score: {sil_score:.3f}, Inertia: {inertia:.2f}")
    
    return results

# original dataset
original_results = apply_kmeans_and_evaluate(X_std, "Original Dataset")

# pca dataset
pca_results = apply_kmeans_and_evaluate(X_pca, "PCA Dataset")

# Plot results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot([r['k'] for r in original_results], [r['silhouette'] for r in original_results], 'o-', label='Original Dataset')
plt.plot([r['k'] for r in pca_results], [r['silhouette'] for r in pca_results], 'o-', label='PCA Dataset')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs. Number of Clusters')
plt.legend()
plt.grid(True)


# find the best on silhouette score
best_k_original = max(original_results, key=lambda x: x['silhouette'])
best_k_pca = max(pca_results, key=lambda x: x['silhouette'])

print("\nBest results:")
print(f"Original Dataset: k={best_k_original['k']}, Silhouette Score={best_k_original['silhouette']:.3f}")
print(f"PCA Dataset: k={best_k_pca['k']}, Silhouette Score={best_k_pca['silhouette']:.3f}")

# determine which is better
if best_k_pca['silhouette'] > best_k_original['silhouette']:
    print("\nPCA dataset is better.")
else:
    print("\nOriginal dataset is better.")




The original dataset's best silhouette score is 0.441. The PCA dataset's best silhoutte score is 0.442. Therefore the PCA dataset results in a better separated and more compact clusters, however the difference between the results is very minor. 

# Task 8 - Clustering Using t-SNE

1. Apply t-SNE (using the exact method) to reduce the dataset to 4 components.
2. Create a 3D scatter plot of the first three t-SNE components.
3. Apply KMeans clustering on the t‐SNE–reduced data using an appropriate number of
clusters (e.g., based on prior optimal k or an elbow method on the t‐SNE output).
4. Evaluate the clustering performance on the t‐SNE–reduced data using metrics such
as the Silhouette Score and compare these results to clustering on the original and
PCA–transformed dataset.
5. Discuss whether the clusters formed on the t-SNE–reduced data are more distinct
and how well they correspond to the known data structure.

In [ ]:
#########
# Task 8 - Clustering Using t-SNE
#
# Purpose: To apply t-SNE to generate a new dataset with 4 components. Compare the clustering results using KMeans on the t-SNE reduced data, the original data and the PCA transformed data.
# Takeaway: The results for the three datasets were:
#           Original Dataset: k=2, Silhouette Score=0.441
#           PCA Dataset: k=2, Silhouette Score=0.484
#           t-SNE Dataset: k=2, Silhouette Score=0.400
#           The PCA dataset has the highest silhouette score, therefore it has the best defined clusters.
#           The t-SNE dataset has the lowest silhouette score, therefore it has the least well defined cluster.
#           Even though t-SNE is a dimensionality reduction technique which theoretically could improve cluster cohesion, the original dataset has only 8 features and therefore may not possess enough dimensions to benefit from this technique.
#########


# compute t-SNE 
tsne = TSNE(n_components=4, method='exact', random_state=42)
X_tsne = tsne.fit_transform(X_std)

# 3D scatter plot of the first three components
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

if 'label' in df.columns:
    labels = df['label'].unique()
    for i, label in enumerate(labels):
        indices = df['label'] == label
        ax.scatter(X_tsne[indices, 0], X_tsne[indices, 1], X_tsne[indices, 2], 
                   label=label, alpha=0.7)
else:
    ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], alpha=0.7)

ax.set_title('3D t-SNE Visualisation')
ax.set_xlabel('t-SNE Component 1')
ax.set_ylabel('t-SNE Component 2')
ax.set_zlabel('t-SNE Component 3')
plt.legend()
plt.tight_layout()
plt.show()

# Apply KMeans clustering on the t-SNE-reduced data
# First, determine the optimal number of clusters using the elbow method
distortions = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_tsne)
    distortions.append(kmeans.inertia_)
    
    labels = kmeans.labels_
    silhouette_avg = silhouette_score(X_tsne, labels)
    silhouette_scores.append(silhouette_avg)


# calc silhouette scores
tsne_results = []
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_tsne)
    silhouette_avg = silhouette_score(X_tsne, labels)
    tsne_results.append({'k': k, 'silhouette': silhouette_avg})

best_k_tsne = max(tsne_results, key=lambda x: x['silhouette'])


print("\nResults:")
print(f"Original Dataset: k={best_k_original['k']}, Silhouette Score={best_k_original['silhouette']:.3f}")
print(f"PCA Dataset: k={best_k_pca['k']}, Silhouette Score={best_k_pca['silhouette']:.3f}")
print(f"t-SNE Dataset: k={best_k_tsne['k']}, Silhouette Score={best_k_tsne['silhouette']:.3f}")

# find the best
best_method = max([
    ('Original', best_k_original['silhouette']),
    ('PCA', best_k_pca['silhouette']),
    ('t-SNE', best_k_tsne['silhouette'])
], key=lambda x: x[1])

print(f"\nThe {best_method[0]} dataset provides better clustering quality with a silhouette score of {best_method[1]:.3f}.")


The t-SNE dataset has the lowest silhouette score, therefore it has the least well defined cluster. t-SNE (t-Distributed Stochastic Neighbour Embedding). Even though t-SNE is a dimensionality reduction technique which theoretically could improve cluster cohesion, the original dataset has only 8 features and therefore may not possess enough dimensions to benefit from this technique.